In [4]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score

dataframe = pd.read_csv('training_set_rel3.tsv', encoding = 'latin-1',sep='\t')
dataframe = dataframe[['essay_id','essay_set','essay','domain1_score']]
dataframe.head()
dataframe.query('essay_set==4').domain1_score.unique()

array([0, 3, 2, 1], dtype=int64)

In [5]:
def normalScore(essay):
    normal = []
    for index,row in essay.iterrows():
        essay_set = row['essay_set']
        score = row['domain1_score']
        if essay_set==1:
            normal.append(score//7)
        if essay_set==2:
            normal.append(score//4)
        if essay_set==3:
            normal.append(score//2)
        if essay_set==4:
            normal.append(score//2)
        if essay_set==5:
            normal.append(score)
        if essay_set==6:
            normal.append(score)
        if essay_set==7:
            normal.append(score//13)
        if essay_set==8:
            normal.append(score//31)
    return essay.assign(normal_score=normal)
dataframe = normalScore(dataframe)
dataframe.head()
dataframe.normal_score.unique()

array([1, 0, 2, 3, 4], dtype=int64)

In [6]:
(dataframe.normal_score==1).sum()

6703

In [7]:
(dataframe.normal_score==0).sum()

3205

In [8]:
import enchant 
import pandas as pd
import spacy
import re
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import urllib.request
from bs4 import BeautifulSoup
print("done")

done


In [9]:
nlp = spacy.load('en_core_web_md')

essays = dataframe

# ----------- Isolate essays from the 6th set ------------ #
essays = essays[(essays['essay_set'] == 6)]
essays.dropna(axis=1, how='all', inplace=True)

# load the enchant dictionary
d = enchant.Dict("en_US")

def getCorrectAndIncorrectSpelling(seriesOfEssays):
    """
    Returns a list of tuples:
    number of mispelled words, number of correctly spelled words 
    """
    _byRow = []

    for essay in seriesOfEssays:
        _individual = []
        for word in essay.split():

            _individual.append(d.check(word))

        _byRow.append((_individual.count(False), _individual.count(True)))

    return _byRow

# ------ Use Enchant to get the number of misspelt and correctly spelt words ------ #
spellings = getCorrectAndIncorrectSpelling(essays['essay'])

# create a list from the tuples
list1, list2 = zip(*spellings)

# assign them to the dataframe
essays = essays.assign(misspelt = list1) 
essays = essays.assign(correct = list2) 
essays.head()



C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,essay_id,essay_set,essay,domain1_score,normal_score,misspelt,correct
8884,14834,6,There were many obstacles that the builders fa...,2,2,6,116
8885,14835,6,"Him from the start, there would have been many...",3,3,11,169
8886,14836,6,The builders of the Empire State Building face...,4,4,3,163
8887,14837,6,In the passage The Mooring Mast by Marcia Amid...,1,1,18,175
8888,14838,6,The builders of the Empire State Building face...,3,3,4,158


In [10]:
tokenizer = RegexpTokenizer(r'\w+')

# Number of words in essay
def get_essay_lengths(regExTokenizer, df):
    """
    Function that gets the number of words in an
    essay, not including punctuation and assigns them to the df
    """
    length = []
    for essay in df['essay']:
        length.append(len(regExTokenizer.tokenize(essay)))
        
    return df.assign(length = length) 

essays = get_essay_lengths(tokenizer, essays)   

# lexical diversity
def get_lexical_diversity(regExTokenizer, df):
    """
    Function that measures lexical diversity which is
    The ratio of total words to unique words
    Then assign that to the df
    """
    ld = []
    for essay in df['essay']:
        
        ld.append(round(len(regExTokenizer.tokenize(essay)) / float(len(set(regExTokenizer.tokenize(essay)))), 2))
    return df.assign(lexical_diversity = ld)

essays = get_lexical_diversity(tokenizer, essays)

# number of sentences
def get_number_of_sentences(df):
    """
    Function that returns the number of sentences
    in the document. Could be useful for run-on sentences
    Assign them to the df
    """
    
    _byRow_sents = []
    for essay in df['essay']:
        sents = []
        parsed_essay = nlp(essay)
        for num, sentence in enumerate(parsed_essay.sents):
            sents.append(sentence)
        
        _byRow_sents.append(len(sents))
        
    return df.assign(n_sentences = _byRow_sents)

essays = get_number_of_sentences(essays)

# Get Parts of Speech
def get_list_of_number_of_pos(df):
    """
    Function that parses the essay for each words POS
    Returns tuples containg for now, nouns, verbs, adverbs and adjectives
    """
    pos = []
    
    for essay in df['essay']:
        parsed_essay = nlp(essay)
        token_pos = [token.pos_ for token in parsed_essay]
        
        pos.append((token_pos.count('NOUN'), token_pos.count('VERB'), token_pos.count('ADV'), token_pos.count('ADJ')))
        
    return pos

pos_list = get_list_of_number_of_pos(essays)
nouns, verbs, adverbs, adjectives = zip(*pos_list)

# Assign them to the dataframe
essays = essays.assign(nouns = nouns)
essays = essays.assign(verbs = verbs)
essays = essays.assign(adverbs = adverbs)
essays = essays.assign(adjectives = adjectives)


# -------- Use Beautiful Soup to scrape for the top 500 SAT words ------- #
maxdf = essays[essays.domain1_score == essays.domain1_score.max()].iloc[0]
from rake_nltk import Rake
r = Rake(max_length=1) # Uses stopwords for english from NLTK, and all puntuation characters.
r.extract_keywords_from_text(maxdf.essay)
ini = r.get_ranked_phrases()
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
fin = set()
for word in ini:
    syns = wordnet.synsets(lemmatizer.lemmatize(word))
    if len(syns) != 0:
        syn = syns[0]
        lemmas = syn.lemmas()
        for lemma in lemmas:
            fin.add(lemma.name())
def wordnettopic(df):
    col = []
    for essay in df['essay']:
        r.extract_keywords_from_text(essay)
        ini = r.get_ranked_phrases()
        tot = 0
        for word in ini:
            if lemmatizer.lemmatize(word) in fin:
                tot+=1
        col.append(tot)
    return df.assign(wordnetscore = col)
essays = wordnettopic(essays)



,essay_id,essay_set,essay,domain1_score,normal_score,misspelt,correct,length,lexical_diversity,n_sentences,nouns,verbs,adverbs,adjectives,wordnetscore,n_top500
8884,14834,6,There were many obstacles that the builders fa...,2,2,6,116,123,1.37,9,28,14,8,10,3,0
8885,14835,6,"Him from the start, there would have been many...",3,3,11,169,180,1.55,10,41,23,12,14,6,0
8886,14836,6,The builders of the Empire State Building face...,4,4,3,163,169,1.62,9,39,23,10,9,24,0
8887,14837,6,In the passage The Mooring Mast by Marcia Amid...,1,1,18,175,199,1.69,11,35,26,7,4,4,0
8888,14838,6,The builders of the Empire State Building face...,3,3,4,158,162,1.74,11,35,26,8,15,8,0


In [12]:
essays.tail()
essays['n_sentences'].corr(essays['domain1_score'])

0.5620257492261873

In [1]:
import pandas as pd
import numpy as np
essays = pd.read_csv('combined.csv',encoding = 'latin-1')
#essays=essays.astype({'avg_distance_bw_centroid_and_points':float,'max_distance_bw_centroid_and_points':float,'min_distance_bw_centroid_and_points':float})
#essays.set_index('essay_id',inplace=True, drop=True)
X = essays.drop(['domain1_score','essay','essay_set','avg_distance_bw_centroid_and_points','max_distance_bw_centroid_and_points','min_distance_bw_centroid_and_points'], axis=1)
y = essays['domain1_score']
X=X.apply(pd.to_numeric,errors='coerce')
X.head()


,num_of_chars,num_of_words,num_of_long_words,num_of_short_words,most_frequent_word_length,average_word_length,num_of_sentences,num_of_long_sents,num_of_short_sents,average_sents_length,...,num_of_pronoun,num_of_proper_noun,num_of_foreign_word,num_of_predeterminer,num_of_possesive_ending,num_of_personal_pronoun,num_of_wh_determiner,num_of_particle,num_of_punctuation,num_of_wh_adverb
0,1875,350,13,151,2,4.357143,16,6,3,21.875000,...,36,5,0,0,0,0,0,16,38,0
1,2288,423,29,175,4,4.408983,21,6,4,20.142857,...,34,3,0,0,0,0,0,10,34,0
2,1541,283,19,126,3,4.445230,17,4,6,16.647059,...,20,1,0,0,0,0,0,10,25,0
3,3165,530,55,189,4,4.971698,29,6,4,18.275862,...,20,30,0,0,0,0,0,23,51,0
4,2569,473,36,205,4,4.431290,30,3,3,15.766667,...,30,2,0,0,0,0,0,20,43,0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)
dt_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=1)

# Fit dt_entropy to the training set
dt_entropy.fit(X_train, y_train)

# Use dt_entropy to predict test set labels
y_pred= dt_entropy.predict(X_test)

# Evaluate accuracy_entropy
accuracy_entropy = cohen_kappa_score(y_pred, y_test)
accuracy_entropy

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:564: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

0.3850096297881447

In [4]:
dt_gini = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)

# Fit dt_entropy to the training set
dt_gini.fit(X_train, y_train)

# Use dt_entropy to predict test set labels
y_pred= dt_gini.predict(X_test)

# Evaluate accuracy_entropy
accuracy_gini = cohen_kappa_score(y_pred, y_test)
accuracy_gini

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:564: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curren

0.4252640762783161

In [16]:
from sklearn.model_selection import cross_val_score
dt_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=2)

# split data set into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

# Compute the array containing the 10-folds CV MSEs
Accuracy_CV_scores = cross_val_score(dt_entropy, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1) 

Accuracy_CV_scores


C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to re

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier

# Set seed for reproducibility
SEED=3

# Instantiate lr
lr = LogisticRegression(random_state=SEED)

# Instantiate knn
knn = KNN(n_neighbors=5)

# Instantiate dt
dt = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=SEED)


# Define the list classifiers
classifiers = [('Logistic Regression', lr), ('K Nearest Neighbours', knn), ('Classification Tree', dt)]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=26)

# Iterate over the pre-defined list of classifiers
for clf_name, clf in classifiers:    
 
    # Fit clf to the training set
    clf.fit(X_train, y_train)    
   
    # Predict y_pred
    y_pred = clf.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_pred, y_test) 
   
    # Evaluate clf's accuracy on the test set
    print('{:s} : {:.3f}'.format(clf_name, accuracy))

Logistic Regression : 0.515
K Nearest Neighbours : 0.594
Classification Tree : 0.612


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [5]:
vc = VotingClassifier(estimators=classifiers)     

# Fit vc to the training set
vc.fit(X_train, y_train)   

# Evaluate the test set predictions
y_pred = vc.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_pred, y_test)
print('Voting Classifier: {:.3f}'.format(accuracy))

NameError: name 'VotingClassifier' is not defined

In [6]:
from sklearn.ensemble import BaggingClassifier

# split data set into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=29)

# Instantiate dt
dt = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_leaf=0.016, random_state=4)

# Instantiate bc
bc = BaggingClassifier(base_estimator=dt, n_estimators=50, random_state=12)

# Fit bc to the training set
bc.fit(X_train, y_train)

# Predict test set labels
y_pred = bc.predict(X_test)

# Evaluate acc_test
acc_test = accuracy_score(y_pred, y_test)
print('Test set accuracy of bc: {:.2f}'.format(acc_test))

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:32: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predi

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_state=random_state)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_state=random_state)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_state=random_state)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release

Test set accuracy of bc: 0.57


C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

# Instantiate dt
dt = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_leaf=0.016, random_state=4)

# Instantiate bc
bc = BaggingClassifier(base_estimator=dt, n_estimators=300, oob_score=True, n_jobs=-1, random_state=13)

# Fit bc to the training set
bc.fit(X_train, y_train)

# Predict test set labels
y_pred = bc.predict(X_test)

# Evaluate acc_test
acc_test = accuracy_score(y_pred, y_test)

# Extract the OOB accuracy from bc
oob_accuracy = bc.oob_score_

print('Test set accuracy of bc: {:.2f}'.format(acc_test)) 

print('OOB accuracy of bc: {:.2f}'.format(oob_accuracy))

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:313: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif isinstance(self.max_features, np.float):
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_state=random_state)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the r

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  random_state=random_state)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the r

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(mask_length, dtype=np.bool)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(mask_length, dtype=np.bool)
C:\Users\aashish\A

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(mask_length, dtype=np.bool)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(mask_length, dtype=np.bool)
C:\Users\aashish\A

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(mask_length, dtype=np.bool)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:609: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(mask_length, dtype=np.bool)
C:\Users\aashish\A

Test set accuracy of bc: 0.54
OOB accuracy of bc: 0.54


In [9]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate rf
rf = RandomForestClassifier(criterion='gini', random_state=2)
            
# Fit rf to the training set    
rf.fit(X_train, y_train)

C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [10]:
rf.get_params()


{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2,
 'verbose': 0,
 'warm_start': False}

In [11]:
from sklearn.model_selection import RandomizedSearchCV

# criterion for information gain
criterion = ['gini', 'entropy']

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'criterion': criterion,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'criterion': ['gini', 'entropy'],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)


# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\aashish\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: Deprecation

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.4min


In [ ]:
rf_random.best_params_


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

rf_random = RandomForestClassifier(criterion='entropy', 
                            n_estimators=1000, 
                            min_samples_leaf=4, 
                            min_samples_split=2, 
                            max_features='auto',
                            max_depth=None,
                            bootstrap=True,
                            random_state=2)

# Fit rf to the training set    
rf_random.fit(X_train, y_train)

# predict
y_pred = rf_random.predict(X_test)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))
print('\n')

In [ ]:
importances = pd.Series(data=rf_random.feature_importances_, index= X_train.columns)

# Sort importances
importances_sorted = importances.sort_values()

# Draw a horizontal barplot of importances_sorted
importances_sorted.plot(kind='barh', color='lightgreen')
plt.title('Features Importances')
plt.show()

KeyError: 1